In [1]:
import torch
from sentence_transformers import SentenceTransformer

import nltk
import pandas as pd
import numpy as np
import sqlite3
import dill as pickle

In [2]:
conn = sqlite3.connect("../cord.db")

In [2]:
df = pd.read_sql_query("SELECT title, abstract, authors, body_text FROM cord19", conn)

In [3]:
df.head()

,title,abstract,authors,body_text
0,Clinical features of culture-proven Mycoplasma...,OBJECTIVE: This retrospective chart review des...,"Madani, Tariq A; Al-Ghamdi, Aisha A",Mycoplasma pneumoniae is a common cause of upp...
1,Nitric oxide: a pro-inflammatory mediator in l...,Inflammatory diseases of the respiratory tract...,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Since its discovery as a biological messenger ...
2,Surfactant protein-D and pulmonary host defense,Surfactant protein-D (SP-D) participates in th...,"Crouch, Erika C",Surfactant protein-D (SP-D) is a member of the...
3,Role of endothelin-1 in lung disease,Endothelin-1 (ET-1) is a 21 amino acid peptide...,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",from Xenopus laevis [16] . ETA receptors in no...
4,Gene expression in epithelial cells in respons...,Respiratory syncytial virus (RSV) and pneumoni...,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",RSV and PVM are viruses of the family Paramyxo...


In [53]:
titles = df['title']
authors = df['authors']
document = df['abstract'] + df['body_text']

In [46]:
# model = SentenceTransformer('monologg/biobert_v1.1_pubmed')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Some weights of the model checkpoint at /home/arnav/.cache/torch/sentence_transformers/monologg_biobert_v1.1_pubmed were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
with open('../data/biobert.pickle', 'rb') as file :
    model = pickle.load(file)

if torch.cuda.is_available():
    model = model.to(torch.device("cuda"))

In [56]:
title_embeddings = model.encode(titles, show_progress_bar=True)

Batches:   0%|          | 0/144 [00:00<?, ?it/s]

In [61]:
document_embeddings = model.encode(document, show_progress_bar=True) # 5 mins

Batches:   0%|          | 0/144 [00:00<?, ?it/s]

In [60]:
with open('data/title_embeddings.pickle', 'wb') as file :
    pickle.dump(title_embeddings, file)

In [63]:
with open('data/document_embeddings.pickle', 'wb') as file :
    pickle.dump(document_embeddings, file)

In [4]:
#for relevant_factors table
df = pd.read_sql_query("SELECT Study, Factors, Excerpt FROM relevant_factors", conn)

study_title = df['Study'].map(str)
study_text = df['Factors'].map(str) + df['Excerpt'].map(str)

study_title_embeddings = model.encode(study_title, show_progress_bar=True) 
study_text_embeddings = model.encode(study_text, show_progress_bar=True)

with open('../data/study_title_embeddings.pickle', 'wb') as file :
    pickle.dump(study_title_embeddings, file)

with open('../data/study_text_embeddings.pickle', 'wb') as file :
    pickle.dump(study_text_embeddings, file)


Batches:   0%|          | 0/10 [00:00<?, ?it/s]

Batches:   0%|          | 0/10 [00:00<?, ?it/s]

In [5]:
#for symptoms table
df = pd.read_sql_query("SELECT Study, Excerpt FROM symptoms", conn)

symptoms_study_title = df['Study'].map(str)
symptoms_study_text = df['Excerpt'].map(str)

symptoms_study_title_embeddings = model.encode(symptoms_study_title, show_progress_bar=True) 
symptoms_study_text_embeddings = model.encode(symptoms_study_text, show_progress_bar=True)

with open('../data/symptoms_study_title_embeddings.pickle', 'wb') as file :
    pickle.dump(symptoms_study_title_embeddings, file)

with open('../data/symptoms_study_text_embeddings.pickle', 'wb') as file :
    pickle.dump(symptoms_study_text_embeddings, file)


Batches:   0%|          | 0/6 [00:00<?, ?it/s]

Batches:   0%|          | 0/6 [00:00<?, ?it/s]